## Final

In [ ]:
# ── helper (put this once, before plot_daily) ─────────────────────────
import plotly.graph_objects as go
import pandas as pd
import pytz

def _stub_fig(sym: str, D) -> go.Figure:
    """Return an empty placeholder figure for a missing day."""
    return (go.Figure()
              .update_layout(title=f"{sym} • {D}  –  (no data)",
                             height=240, width=700, template="plotly_white",
                             xaxis_visible=False, yaxis_visible=False,
                             annotations=[dict(text="(no rows in 04-20 ET)",
                                               x=0.5, y=0.5, showarrow=False,
                                               font=dict(color="grey", size=14))]))


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# Live HF Bollinger Notebook – COMPLETE VERSION (updated Aug 2025)
# ---------------------------------------------------------------------------
# • Back‑fills **yesterday _and today‑to‑now_** (1‑min & 1‑sec) via Polygon REST
# • Builds full indicator set required by the 4‑panel plot_daily
# • Adds 40‑sec overlays, 10‑min SMA, 5‑min Kalman forecast
# • **New** ▪ raw + smoothed *upper* 40‑s band on Panels 1 & 4
# • **New** ▪ colour‑coded intraday volume bars (secondary‑Y) on Panel 1
# • Shows a live‑updating figure for **today** (pre‑, RTH, post‑) per ticker
# ---------------------------------------------------------------------------

# === Cell 1 · install + imports ================================================
# !pip install -q polygon-api-client pykalman nest_asyncio plotly talib-binary

import nest_asyncio, asyncio, warnings, pytz
import numpy as np, pandas as pd, talib
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from polygon import RESTClient, WebSocketClient
from polygon.websocket.models import Feed, Market
from pykalman import KalmanFilter
from datetime import datetime, timedelta, time as _t
from collections import defaultdict
from typing import List, Optional

nest_asyncio.apply()
warnings.filterwarnings("ignore", "Series.fillna")  # silence pandas future‑warn

API_KEY  = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"          # ← replace with your real key
TICKERS  = ["AAPL","NVDA", "MU","AMD","GOOG"]            # demo   – add more if you like

# TICKERS  = ["IBP","AAPL","TSLA","NVDA", "MU","AMD","AMZN","PLTR","GOOG"]            # demo   – add more if you like
NY       = pytz.timezone("America/New_York")

CFG = {"displayModeBar": True, "scrollZoom": False}  # plotly UI prefs

In [ ]:


# === Cell 2 · helpers ==========================================================

def _tz(obj):
    """Vector‑safe convert / localise to US‑Eastern."""
    ts = pd.to_datetime(obj, errors="coerce")
    return ts.dt.tz_convert(NY) if ts.dt.tz is not None else ts.dt.tz_localize(NY)


def _row_bar(bar):
    """Polygon Agg / EquityAgg → tidy dict row (works for REST *and* WS)."""
    ts_ms = getattr(bar, "timestamp", None) or getattr(bar, "t", None) or getattr(bar, "start_timestamp", None)
    ts    = pd.to_datetime(ts_ms, unit="ms", utc=True).tz_convert(NY)
    o, h, l, c, v = (getattr(bar, f, None) or getattr(bar, f[0], None) for f in ("open", "high", "low", "close", "volume"))
    return dict(open=o, high=h, low=l, close=c, volume=v, TIME_EST=ts)


In [ ]:

# === Cell 3 · REST back‑fill (yesterday + today) ==============================
from pandas.tseries.offsets import BDay
rest      = RESTClient(API_KEY)
raw_min   = defaultdict(pd.DataFrame)
raw_sec   = defaultdict(pd.DataFrame)

# yday  = (datetime.now(tz=NY) - timedelta(days=1)).strftime("%Y-%m-%d")
# today =  datetime.now(tz=NY).strftime("%Y-%m-%d")





today_dt = pd.Timestamp.now(tz=NY).normalize()
yday_dt  = today_dt - BDay(1)        # ← skips weekends & holidays

today = today_dt.strftime("%Y-%m-%d")
yday  = yday_dt.strftime("%Y-%m-%d")
for sym in TICKERS:
    for day in (yday, today):
        # 1‑minute bars ----------------------------------------------------
        # mbars = rest.list_aggs(sym, 1, "minute", day, day, adjusted=True, limit=50_000)
        
        
                # 1-minute bars ----------------------------------------------------
        mbars = rest.list_aggs(
            sym, 1, "minute",
            day, day,
            adjusted=True,
            limit=50_000,
            # include_otc=True      # ← add this
        )

        raw_min[sym] = pd.concat([raw_min[sym], pd.DataFrame([_row_bar(b) for b in mbars])], ignore_index=True)

        # 1‑second bars ----------------------------------------------------
        sbars = rest.list_aggs(sym, 1, "second", day, day, adjusted=True, limit=50_000)
        raw_sec[sym] = pd.concat([raw_sec[sym], pd.DataFrame([_row_bar(b) for b in sbars])], ignore_index=True)

print("REST back‑fill →", {k: len(v) for k, v in raw_min.items()})

REST back‑fill → {'AAPL': 1316, 'NVDA': 1449, 'MU': 994, 'AMD': 1425, 'GOOG': 904}


In [ ]:
# counts you already print for 1-minute bars
print("minute bars:", {k: len(v) for k, v in raw_min.items()})

# do the same for 1-second bars
print("second bars:", {k: len(v) for k, v in raw_sec.items()})


minute bars: {'AAPL': 1316, 'NVDA': 1449, 'MU': 994, 'AMD': 1425, 'GOOG': 904}
second bars: {'AAPL': 34639, 'NVDA': 42139, 'MU': 21643, 'AMD': 37692, 'GOOG': 20628}


In [ ]:
# ──────────────────────────────────────────────────────────────────────────
#  ZigZag swing-pivot detector
#     • threshold :   swing size  (percent if pct=True, else absolute units)
#     • pct       :   interpret threshold as % of last pivot (True) or abs (False)
#     • causal    :   True ⇒ last unfinished pivot is discarded (back-test safe)
#     • min_bars  :   minimum bars that must elapse before a new pivot is allowed
#                     (0 disables the time-gap guard)
#  RETURNS        :   pd.Series indexed by pivot datetime, value = pivot level
# ──────────────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np

def _zigzag_pivots(series: pd.Series,
                   threshold: float = 1.5,
                   pct: bool = False,
                   causal: bool = True,
                   min_bars: int = 0) -> pd.Series:
    s = pd.Series(series, copy=False).astype(float).dropna()
    if len(s) < 3:
        return pd.Series(dtype=float)

    piv_idx, piv_val = [s.index[0]], [s.iloc[0]]
    trend           = 0          # 0 = unknown, 1 = up, −1 = down
    bars_since_piv  = 0

    def hit(a, b):
        return abs((b - a) / a) >= threshold if pct else abs(b - a) >= threshold

    for idx, val in s.iloc[1:].items():
        bars_since_piv += 1
        last = piv_val[-1]

        if trend == 0:                              # first direction not set
            if hit(last, val):
                trend = 1 if val > last else -1
                piv_idx[-1], piv_val[-1] = idx, val
                bars_since_piv = 0

        elif trend == 1:                            # currently rising
            if val > last:                          # higher high → extend
                piv_idx[-1], piv_val[-1] = idx, val
                bars_since_piv = 0
            elif hit(last, val) and bars_since_piv >= min_bars:  # reversal
                trend = -1
                piv_idx.append(idx); piv_val.append(val)
                bars_since_piv = 0

        else:                                       # currently falling
            if val < last:                          # lower low → extend
                piv_idx[-1], piv_val[-1] = idx, val
                bars_since_piv = 0
            elif hit(last, val) and bars_since_piv >= min_bars:  # reversal
                trend = 1
                piv_idx.append(idx); piv_val.append(val)
                bars_since_piv = 0

    # discard unfinished last pivot for back-test safety
    if causal and len(piv_idx) > 1:
        piv_idx, piv_val = piv_idx[:-1], piv_val[:-1]

    return pd.Series(piv_val, index=piv_idx, name="pivot")


In [ ]:
def _mpc_olive(x0: float, v0: float,
               H: int = 5, lam: float = 0.05, umax: float = 0.05):
    """
    Tiny MPC forecaster for the olive series.
    Returns an ndarray length H with predicted levels.
    """
    A = np.array([[1., 1.],
                  [0., 1.]])
    B = np.array([[0.5],
                  [1. ]])

    u = cp.Variable(H)
    s = np.array([[x0], [v0]])
    states = []
    for k in range(H):
        s = A @ s + B * u[k]
        states.append(s[0, 0])

    x_pred = cp.hstack(states)

    # naïve target = straight-line extrapolation of last slope
    y_star = x0 + v0 * np.arange(1, H + 1)

    obj = cp.Minimize(cp.sum_squares(x_pred - y_star) +
                      lam * cp.sum_squares(u))
    prob = cp.Problem(obj, [cp.abs(u) <= umax])
    prob.solve(solver="OSQP", warm_start=True)

    return np.asarray(x_pred.value, dtype=float)


In [ ]:
# ---------------------------------------------------------------------
# helper: attempt several solvers, stay quiet
# ---------------------------------------------------------------------
def _solve_qp(prob) -> bool:
    """Try OSQP → ECOS → SCS.  Return True on (near-)optimal solution."""
    for s in ("OSQP", "ECOS", "SCS"):
        try:
            prob.solve(solver=s,
                       warm_start=True,   # reuse last solution, faster
                       verbose=False)     # silence solver output
            if prob.status in ("optimal", "optimal_inaccurate"):
                return True
        except (cp.SolverError, ValueError):
            pass                         # try the next solver
    return False                         # none succeeded


# ---------------------------------------------------------------------
# MPC forecaster: always returns something (never raises SolverError)
# ---------------------------------------------------------------------
def _mpc_olive(x0: float, v0: float,
               *, H: int = 5, lam: float = 0.05, umax: float = 0.05) -> np.ndarray:
    """Return H-step forecast for the olive series."""
    # state-space matrices (Δt = 1 min)
    A = np.array([[1., 1.],
                  [0., 1.]])
    B = np.array([[0.5],
                  [1. ]])

    u    = cp.Variable(H)
    s    = np.array([[x0], [v0]])
    x_lv = []                              # predicted levels
    for k in range(H):
        s  = A @ s + B * u[k]
        x_lv.append(s[0, 0])

    y_star = x0 + v0 * np.arange(1, H + 1)  # naive straight-line target
    obj    = cp.Minimize(cp.sum_squares(cp.hstack(x_lv) - y_star)
                         + lam * cp.sum_squares(u))
    prob   = cp.Problem(obj, [cp.abs(u) <= umax])

    if not _solve_qp(prob):
        # ---- fallback: linear extrapolation (never crashes) ------------
        return x0 + v0 * np.arange(1, H + 1)

    # convert cvxpy expressions → float array
    return np.array([float(lv.value) for lv in x_lv])


In [ ]:
# === Cell 4 · enrich_signals (≤3 trades/day) ==============================
import numpy as np, pandas as pd, talib, pytz, cvxpy as cp
from datetime import time as _t
from pykalman import KalmanFilter
from pandas import Series

NY = pytz.timezone("America/New_York")


# ──────────────────────────────────────────────────────────────────────────
def enrich_signals(df_min: pd.DataFrame,
                   df_sec: pd.DataFrame,
                   *, in_start: _t = _t(4), in_end: _t = _t(20),
                   cooldown: int = 20) -> pd.DataFrame:

    if df_min.empty:
        return pd.DataFrame()

    # --- base 1-minute frame --------------------------------------------
    d = df_min.copy()
    d.TIME_EST = pd.to_datetime(d.TIME_EST, utc=True).dt.tz_convert(NY)
    c, v = d.close.astype(float), d.volume.astype(float)

    # --- 1-min indicators ------------------------------------------------
    d["BB_up"], _, d["BB_dn"] = talib.BBANDS(c, 20, 2, 2)
    for name, period in [("EMA9", 9), ("EMA20", 20), ("EMA50", 50), ("EMA200", 200)]:
        fn = talib.EMA if name.startswith("EMA") else talib.SMA
        d[name] = fn(c, period)
    macd, macd_s, macd_h = talib.MACD(c, 12, 26, 9)
    d["MACD"], d["MACD_Signal"], d["MACD_hist"] = macd, macd_s, macd_h
    d["ATR"] = talib.ATR(d.high, d.low, c, 14)

    mask = d.TIME_EST.dt.time.between(in_start, in_end)
    d.loc[mask, "VWAP_Cum"] = (
        (d.loc[mask, "close"] * d.loc[mask, "volume"])
        .groupby(d.loc[mask, "TIME_EST"].dt.date).cumsum()
        / d.loc[mask, "volume"].groupby(d.loc[mask, "TIME_EST"].dt.date).cumsum()
    )
    for n in (5, 20, 50, 200):
        d[f"SMA{n}"] = talib.SMA(c, n)

    # =====================================================================
    # SECOND-LEVEL OVERLAYS (40 s)  → resampled to 1-min
    # =====================================================================
    if not df_sec.empty:
        s = df_sec.copy()
        s.TIME_EST = pd.to_datetime(s.TIME_EST, utc=True).dt.tz_convert(NY)
        sc = s.close.astype(float)

        up40, _, dn40 = talib.BBANDS(sc, 40, 2.2, 2.2)
        s["BB_UP_sec"], s["BB_DN_sec"] = up40, dn40
        for col in ("BB_UP_sec", "BB_DN_sec"):
            s[col + "_smooth"] = s[col].ewm(span=5, adjust=False).mean()

        m40, ms40, mh40 = talib.MACD(sc, 12, 26, 9)
        s[["MACD_sec", "MACD_Signal_sec", "MACD_hist_sec"]] = np.column_stack([m40, ms40, mh40])

        cols = [c for c in s.columns if c.endswith("_sec") or c.endswith("_smooth")]
        s1 = (s.set_index("TIME_EST")[cols].resample("1min").last().reset_index())
        d = pd.merge_asof(d.sort_values("TIME_EST"), s1.sort_values("TIME_EST"),
                          on="TIME_EST", direction="backward",
                          tolerance=pd.Timedelta("59s"))
        d.rename(columns=lambda col: col.replace("_sec", "_sec_on_1m"), inplace=True)

        # 10-min SMA of 40-s bands
        d["BB_DN_sec_10m_on_1m"] = d["BB_DN_sec_on_1m"].rolling(10, 10).mean()
        d["BB_UP_sec_10m_on_1m"] = d["BB_UP_sec_on_1m"].rolling(10, 10).mean()

        # -------- amplify + z-score (olive raw) --------------------------
        AMP = 2.0
        d["BB_UP_sec_on_1m_amp"] = d["BB_UP_sec_on_1m"] * AMP
        z_mu = d["BB_UP_sec_on_1m_amp"].rolling(40, min_periods=8).mean()
        z_sd = d["BB_UP_sec_on_1m_amp"].rolling(40, min_periods=8).std(ddof=0)
        d["BB_UP_sec_on_1m_amp_z"] = (d["BB_UP_sec_on_1m_amp"] - z_mu) / z_sd

        # -------- MPC one-step forecast ----------------------------------
        d["olive_mpc_1"] = np.nan
        if len(d) >= 3 and not np.isnan(d["BB_UP_sec_on_1m_amp_z"].iat[-1]):
            x_t = d["BB_UP_sec_on_1m_amp_z"].iat[-1]
            v_t = x_t - d["BB_UP_sec_on_1m_amp_z"].iat[-2]
            x_pred = _mpc_olive(x_t, v_t, H=5)   # 5-bar trajectory
            d.loc[d.index[-1], "olive_mpc_1"] = x_pred[0]   # +1 min
            d.loc[d.index[-1], "olive_mpc_5"] = x_pred[-1]  # +5 min

            # d.loc[d.index[-1], "olive_mpc_1"] = _mpc_olive(x_t, v_t)[0]
            
        # after: d.loc[d.index[-1], "olive_mpc_1"] = ...
        if len(d) >= 2:
            # actual olive value one bar *after* the forecast was made
            d.loc[d.index[-2], "olive_mpc_err"] = (
                d["BB_UP_sec_on_1m_amp_z"].iat[-1]   # t   actual
                - d["olive_mpc_1"].iat[-2]           # t-1 forecast
            )


        # -------- ZigZag on the same z-score -----------------------------
        PIVOT, MINB = 1.5, 5
        zig = _zigzag_pivots(d["BB_UP_sec_on_1m_amp_z"],
                             threshold=PIVOT, pct=False,
                             causal=True, min_bars=MINB)
        # ------------------------------------------------------------------
# After d["BB_UP_sec_on_1m_amp_z"] is ready
# ------------------------------------------------------------------
        d["olive_mpc_1"] = np.nan

        for i in range(2, len(d)):          # start at the 3rd bar
            x_t = d["BB_UP_sec_on_1m_amp_z"].iat[i-1]
            v_t = x_t - d["BB_UP_sec_on_1m_amp_z"].iat[i-2]
            d["olive_mpc_1"].iat[i] = _mpc_olive(x_t, v_t)[0]

        d["BB_UP_sec_on_1m_amp_z_zz"] = zig.reindex(d.index).interpolate()

        piv_lo = zig[(zig.shift(1) > zig) & (zig.shift(-1) > zig)].index
        piv_hi = zig[(zig.shift(1) < zig) & (zig.shift(-1) < zig)].index
        mask_lo, mask_hi = d.index.isin(piv_lo), d.index.isin(piv_hi)

        trend = (d["SMA50"] >= d["SMA200"]) & (d["BB_UP_sec_on_1m"] >= d["VWAP_Cum"])

        d["Entry_Signal"] = (mask_lo & trend).astype(int)
        d["Exit_Signal"]  = (mask_hi & trend).astype(int)

        # -------- early flip using MPC -----------------------------------
        olive, fore = d["BB_UP_sec_on_1m_amp_z"], d["olive_mpc_1"]
        flip_up   = (fore > olive) & (olive.shift(1) >= olive)
        flip_down = (fore < olive) & (olive.shift(1) <= olive)
        d["Entry_Signal_MPC"] = (flip_up   & trend).astype(int)
        d["Exit_Signal_MPC"]  = (flip_down & trend).astype(int)

        # ---- extra low-pass + Kalman for plotting (optional) ------------
        lp = d["BB_UP_sec_on_1m_amp_z"].ewm(span=20, adjust=False).mean()
        d["BB_UP_sec_on_1m_amp_z_lp"] = lp
        if d["BB_DN_sec_on_1m"].notna().sum() > 20:
            y = d["BB_DN_sec_on_1m"].ffill().values
            kf = KalmanFilter(transition_matrices=[[1, 1], [0, 1]],
                              observation_matrices=[[1, 0]],
                              transition_covariance=np.diag([1e-4, 1e-5]),
                              observation_covariance=1e-3)
            lvl, slp = kf.filter(y)[0].T
            d["BB_dn_kalman"] = lvl + slp * 5
    else:
        # no second-level data
        cols = ["BB_UP_sec_on_1m_amp", "BB_UP_sec_on_1m_amp_z",
                "BB_UP_sec_on_1m_amp_z_zz", "olive_mpc_1"]
        for col in cols:
            d[col] = np.nan
        d["Entry_Signal"] = d["Exit_Signal"] = 0
        d["Entry_Signal_MPC"] = d["Exit_Signal_MPC"] = 0

    # ---------- risk / reward calc & daily quota ------------------------
    stop   = c - d.ATR
    target = c + d.ATR * 2
    d["RR"] = (target - c) / (c - stop)
    d["date"] = d.TIME_EST.dt.date
    keep = (d[d.Entry_Signal == 1]
            .sort_values(["date", "RR"], ascending=[True, False])
            .groupby("date", group_keys=False).head(3).index)
    d.loc[(d.Entry_Signal == 1) & ~d.index.isin(keep), "Entry_Signal"] = 0
    d.drop(columns="date", inplace=True)

    # ---------- pair entries to the next exits --------------------------
               # ← and use it

    exit_idx = d.index[d.Exit_Signal == 1]
    next_exit = [exit_idx[exit_idx > ent][0] if len(exit_idx[exit_idx > ent]) else None
                 for ent in d.index[d.Entry_Signal == 1]]
    valid_exits = [i for i in next_exit if i is not None]   # ← add this
    d.loc[valid_exits, "Exit_Signal"] = 1 
    # d["Exit_Signal"] = 0
    # # d.loc[next_exit, "Exit_Signal"] = 1
    # print(dm["olive_mpc_1"].dropna().head())
    # print(dm["olive_mpc_1"].dropna().tail())


    return d


In [ ]:
# helper goes somewhere once, e.g. right above plot_daily
def _stub_fig(sym: str, D) -> go.Figure:
    return (go.Figure()
              .update_layout(title=f"{sym} • {D}  –  (no data)",
                             height=240, width=700, template="plotly_white",
                             xaxis_visible=False, yaxis_visible=False,
                             annotations=[dict(text="(no rows in 04-20 ET)",
                                               x=0.5, y=0.5,
                                               showarrow=False,
                                               font=dict(color="grey", size=14))]))


In [ ]:
from datetime import timedelta, time as _t
from typing import List
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np, pandas as pd           # ← already imported earlier

# ───────────────────────────────────────────────────────────────
# plot helpers + *fixed* plot_daily
# ───────────────────────────────────────────────────────────────
def _band(fig, x, y, name, color, *, row=1, col=1, fill=None, dash=None, width=2):
    fig.add_trace(go.Scatter(
        x=x, y=y, name=name,
        line=dict(color=color, width=width, dash=dash),
        fill="tonexty" if fill else None,
        fillcolor=fill or "rgba(0,0,0,0)",
        legendgroup=name.split()[0]
    ), row=row, col=col)


def plot_daily(sym: str, df: pd.DataFrame, *,
               height: int = 800, width: int = 1100,
               start: _t = _t(4, 0), end: _t = _t(20, 0)) -> List[go.Figure]:

    if df.empty:
        # return []
        return None, None

    figs: List[go.Figure] = []

    for D, dm in df.groupby(df.TIME_EST.dt.date):

        # keep only rows inside 04:00-20:00
        dm = dm.loc[dm.TIME_EST.dt.time.between(start, end)]

        # if the mask removed everything → placeholder figure
        if dm.empty:
            figs.append(_stub_fig(sym, D))
            continue

        # ════════════════════════════════════════════════════════
        # normal figure (unchanged layout)
        # ════════════════════════════════════════════════════════
        dm = dm.copy()
        dm["volume_color"] = np.where(dm.close > dm.close.shift(1), "green",
                                      np.where(dm.close < dm.close.shift(1), "red", "gray"))

        fig = make_subplots(rows=5, cols=1, shared_xaxes=True,
                            row_heights=[.45, .20, .20, .15, .20],
                            vertical_spacing=.02,
                            specs=[[{"secondary_y": True}], [{},], [{},], [{},], [{}]])

        # panel-1 candles
        fig.add_trace(go.Candlestick(x=dm.TIME_EST, open=dm.open, high=dm.high,
                                     low=dm.low, close=dm.close,
                                     increasing_line_color="green",
                                     decreasing_line_color="red",
                                     showlegend=False), 1, 1)

        # Bollinger bands (panel-1)
        _band(fig, dm.TIME_EST, dm.BB_dn, "BB_dn (1m)", "#60a5fa")
        _band(fig, dm.TIME_EST, dm.BB_up, "BB_up (1m)", "#60a5fa",
              fill="rgba(59,130,246,0.10)")
        _band(fig, dm.TIME_EST, (dm.BB_up+dm.BB_dn)/2, "BB_mid (1m)",
              "#60a5fa", dash="dot")

        # 40-s smooth bands
        if "BB_DN_sec_smooth_on_1m" in dm:
            _band(fig, dm.TIME_EST, dm.BB_DN_sec_smooth_on_1m,
                  "BB_dn (40s)", "#f59e0b")
            _band(fig, dm.TIME_EST, dm.BB_UP_sec_smooth_on_1m,
                  "BB_up (40s)", "#f59e0b",
                  fill="rgba(234,179,8,0.12)")

        # raw 40-s bands on panel-4
        if "BB_UP_sec_on_1m" in dm:
            _band(fig, dm.TIME_EST, dm.BB_UP_sec_on_1m,
                  "BB_up (raw 40s)", "#f59e0b", row=4, col=1)

        if "BB_UP_sec_10m_on_1m" in dm:
            _band(fig, dm.TIME_EST, dm.BB_UP_sec_10m_on_1m,
                  "BB_up SMA10", "darkblue", dash="dash", row=4, col=1)

        if "BB_dn_kalman" in dm:
            _band(fig, dm.TIME_EST, dm.BB_dn_kalman,
                  "Kalman (5m)", "pink", dash="dash", row=4, col=1)

        # volume (secondary-y)
        fig.add_trace(go.Bar(x=dm.TIME_EST, y=dm.volume, name="Vol",
                             marker_color=dm.volume_color,
                             opacity=0.25, marker_line_width=0),
                      1, 1, secondary_y=True)

        # trend lines
        fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm.VWAP_Cum,
                                 name="VWAP", line=dict(color="purple", width=2)), 1, 1)
        if "EMA9" in dm:
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm.EMA9,
                                     name="EMA9", line=dict(color="gold", width=2)), 1, 1)

        # entry / exit markers
        fig.add_trace(go.Scatter(x=dm.TIME_EST[dm.Entry_Signal == 1],
                                 y=dm.low[dm.Entry_Signal == 1],
                                 mode="markers",
                                 marker=dict(symbol="triangle-up", size=12,
                                             color="#10b981", line=dict(color="black")),
                                 name="Entry"), 1, 1)
        fig.add_trace(go.Scatter(x=dm.TIME_EST[dm.Exit_Signal == 1],
                                 y=dm.high[dm.Exit_Signal == 1],
                                 mode="markers",
                                 marker=dict(symbol="triangle-down", size=12,
                                             color="#ef4444", line=dict(color="black")),
                                 name="Exit"), 1, 1)

        # panel-2 MACD
        fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm.MACD,
                                 name="MACD", line=dict(color="orange", width=2)), 2, 1)
        fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm.MACD_Signal,
                                 name="Signal", line=dict(color="blue", width=1)), 2, 1)
        fig.add_trace(go.Bar(x=dm.TIME_EST, y=dm.MACD_hist,
                             marker_color=np.where(dm.MACD_hist >= 0, "green", "red"),
                             showlegend=False), 2, 1)
        fig.add_shape(type="line", xref="paper", x0=0, x1=1,
                      yref="y2", y0=0, y1=0,
                      line=dict(color="gray", width=1))

        # panel-3 (40-s MACD) — unchanged
        if "MACD_sec_on_1m" in dm:
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm.MACD_sec_on_1m,
                                     name="MACD (40s)",
                                     line=dict(color="orange", dash="dash")), 3, 1)
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm.MACD_Signal_sec_on_1m,
                                     name="Signal (40s)",
                                     line=dict(color="blue", dash="dash")), 3, 1)
            fig.add_trace(go.Bar(x=dm.TIME_EST, y=dm.MACD_hist_sec_on_1m,
                                 marker_color=np.where(dm.MACD_hist_sec_on_1m >= 0,
                                                       "green", "red"),
                                 showlegend=False), 3, 1)
            fig.add_shape(type="line", xref="paper", x0=0, x1=1,
                          yref="y3", y0=0, y1=0,
                          line=dict(color="gray", width=1))

        # panel-4 slow trends
        for col, colr in [("SMA50", "green"),
                          ("SMA200", "blue"),
                          ("VWAP_Cum", "purple")]:
            if col in dm:
                fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm[col],
                                         name=col,
                                         line=dict(color=colr, width=2.5)), 4, 1)
        fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["EMA9"],
                                 name="EMA9", line=dict(color="gold", width=2.5)), 4, 1)

        # # panel-5 extra signals
        # fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["BB_UP_sec_on_1m_amp_z_zz"],
        #                          name="zigzag_BB_UP_amp_z",
        #                          line=dict(color="tomato", width=2.5)), 5, 1)
        # fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["BB_UP_sec_on_1m_amp_z"],
        #                          name="z_BB_UP_sec_on_1m_am",
        #                          line=dict(color="olive", width=1)), 5, 1)

        # fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["olive_mpc_1"],name="MPC 1-step",mode="lines",
        #             line=dict(color="black", width=1, dash="dot")),
        #     row=5, col=1
        # )
        # fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["olive_mpc_5"],name="MPC 5-step",mode="lines",line=dict(color="gold", width=1, dash="dot")),
        #     row=5, col=1
        # )
        # panel-5 extra signals
        if "olive_mpc_1" in dm:
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["olive_mpc_1"],
                                    name="MPC 1-step", mode="lines",
                                    line=dict(color="black", width=1, dash="dot")),
                        row=5, col=1)

        if "olive_mpc_5" in dm:
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["olive_mpc_5"],
                                    name="MPC 5-step", mode="lines",
                                    line=dict(color="gold", width=1, dash="dot")),
                        row=5, col=1)

        if "BB_UP_sec_on_1m_amp_z_zz" in dm:
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["BB_UP_sec_on_1m_amp_z_zz"],
                                    name="zigzag_BB_UP_amp_z",
                                    line=dict(color="tomato", width=2.5)),
                        row=5, col=1)

        if "BB_UP_sec_on_1m_amp_z" in dm:
            fig.add_trace(go.Scatter(x=dm.TIME_EST, y=dm["BB_UP_sec_on_1m_amp_z"],
                                    name="z_BB_UP_sec_on_1m_am",
                                    line=dict(color="olive", width=1)),
                        row=5, col=1)


        





        # v-lines (***this block is now *inside* the loop***)
        entry_times = dm.TIME_EST[dm.Entry_Signal == 1]
        exit_times  = dm.TIME_EST[dm.Exit_Signal  == 1]
        for t in entry_times:
            fig.add_vline(x=t, line_width=2, line_color="darkcyan",
                          row="all", yref="paper")
        for t in exit_times:
            fig.add_vline(x=t, line_width=2, line_color="red",
                          row="all", yref="paper")

        # layout
        fig.update_layout(height=height, width=width, template="plotly_white",
                          title=f"{sym} • {D}", hovermode="x unified",
                          showlegend=True)
        fig.update_xaxes(rangeslider_visible=False, range=[
            pd.Timestamp(D, tz=df.TIME_EST.dt.tz).replace(hour=start.hour,
                                                         minute=start.minute),
            pd.Timestamp(D, tz=df.TIME_EST.dt.tz).replace(hour=end.hour,
                                                         minute=end.minute)])
        fig.update_yaxes(showgrid=True)

        figs.append(fig)

    # ------------------------------------------------------------------
    # ensure yesterday + today always present (stub if empty)
    # ------------------------------------------------------------------
    tz_now    = pd.Timestamp.now(df.TIME_EST.dt.tz)
    today     = tz_now.date()
    # yesterday = today - timedelta(days=1)
    by_date   = {pd.to_datetime(f.layout.title.text.split("•")[1]).date(): f
                 for f in figs}

    # if yesterday not in by_date:
    #     figs.insert(0, _stub_fig(sym, yesterday))
    if today not in by_date:
        figs.append(_stub_fig(sym, today))

    return [figs[0], figs[-1]]


In [ ]:
# make a timezone-aware "today 04:00" reference  ────────────────
four_am = pd.Timestamp.now(tz=NY).normalize().replace(hour=4)

if raw_min[sym].empty or raw_min[sym].TIME_EST.min() > four_am:
    # build minute bars from seconds → guaranteed pre- and post-market
    df_sec = raw_sec[sym].set_index("TIME_EST").sort_index()
    raw_min[sym] = (
        df_sec
        .resample("1min", label="left", closed="left")
        .agg({"open":"first", "high":"max", "low":"min",
              "close":"last", "volume":"sum"})
        .dropna(subset=["close"])
        .reset_index()
    )


In [ ]:
processed_min = {s: enrich_signals(raw_min[s], raw_sec[s]) for s in TICKERS}

for s in TICKERS:
    y_fig, t_fig = plot_daily(s, processed_min[s])
    # y_fig.show(config=CFG)   # yesterday (stub if no rows)
    t_fig.show(config=CFG)   # today, auto-updating


/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:83: UserWarning:

Converting A to a CSC (compressed sparse column) matrix; may take a while.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:113: UserWarning:

Converting P to a CSC (compressed sparse column) matrix; may take a while.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/cvxpy/problems/problem.py:1539: UserWarning:

Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:83: UserWarning:

Converting A to a CSC (compressed sparse column) matrix; may take a while.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:113: UserWarning:

Converting P to a CSC (compressed sparse column) matri

In [ ]:

# # === Cell 6 · build processed_min & snapshots ===============================

# processed_min = {s: enrich_signals(raw_min[s], raw_sec[s]) for s in TICKERS}
# print({s: len(df) for s, df in processed_min.items()})

# for s in TICKERS:
#     figs = plot_daily(s, processed_min[s])
#     if len(figs) > 1:
#         figs[0].show(config=CFG)   # yesterday snapshot
#     figs[-1].show(config=CFG)      # today (will update live)

In [ ]:
last_dt = processed_min[s].index.max()
print(f"[{s}]  last timestamp in data : {last_dt}")


[GOOG]  last timestamp in data : 903


In [ ]:
# === Cell 7 · live widgets & WebSocket ==================================
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from polygon.websocket.models import Feed, Market
from polygon import WebSocketClient

# ── 0) make sure these dicts exist before we start the stream ───────────
raw_min      = defaultdict(pd.DataFrame)   # 1-min bars per symbol
raw_sec      = defaultdict(pd.DataFrame)   # 1-sec bars per symbol
processed_min = {}                         # enriched minutes
live_figs     = {}                         # plotly figs keyed by symbol

# (if you already built static figures earlier, populate live_figs now)
# for s in TICKERS:
#     y_fig, t_fig = plot_daily(s, processed_min.get(s, pd.DataFrame()))
#     y_fig.show(config=CFG)
#     t_fig.show(config=CFG)
#     live_figs[s] = t_fig                   # save today’s figure handle
    
    
for s in TICKERS:
    y_fig, t_fig = plot_daily(s, processed_min.get(s, pd.DataFrame()))
    if all([y_fig, t_fig]):
        y_fig.show(config=CFG)
        t_fig.show(config=CFG)
        live_figs[s] = t_fig



# ────────────────────────────────────────────────────────────────────────
async def ws_handle(msgs):
    """Callback for every websocket message burst (1-n messages)."""
    for m in msgs:
        sym = getattr(m, "sym", None) or getattr(m, "symbol", None)
        if sym is None or sym not in TICKERS:
            continue

        etype = getattr(m, "event_type", "")
        row   = _row_bar(m)

        # ─ 1) accumulate raw bars ───────────────────────────────────────
        if etype == "A":          # 1-second aggregate
            raw_sec[sym] = pd.concat([raw_sec[sym], pd.DataFrame([row])],
                                     ignore_index=True)

        elif etype == "AM":       # 1-minute aggregate
            raw_min[sym] = pd.concat([raw_min[sym], pd.DataFrame([row])],
                                     ignore_index=True)

            # ─ 2) enrich & store latest minute frame ───────────────────
            processed_min[sym] = enrich_signals(raw_min[sym], raw_sec[sym])
            df = processed_min[sym]

            # ─ 3) update live figure in-place ──────────────────────────
            lf = live_figs.setdefault(sym, plot_daily(sym, df)[1])  # create if missing

            # candle trace (index 0) ------------------------------------
            lf.data[0].update(
                x=df.TIME_EST,
                open=df.open, high=df.high, low=df.low, close=df.close
            )
            # volume trace (index 1) ------------------------------------
            lf.data[1].update(
                x=df.TIME_EST, y=df.volume,
                marker_color=np.where(
                    df.close > df.close.shift(1), "green",
                    np.where(df.close < df.close.shift(1), "red", "gray")
                )
            )

            # add / update Kalman trace if present ----------------------
            if "BB_dn_kalman" in df.columns:
                if len(lf.data) < 3:                    # add once
                    lf.add_trace(
                        go.Scatter(name="Kalman (5 min)",
                                   line=dict(color="pink", dash="dash")),
                        row=4, col=1
                    )
                lf.data[-1].update(x=df.TIME_EST, y=df.BB_dn_kalman)

            # ─ 4) keep the last 390 mins visible ----------------------
            n_rows = len(df)
            start  = max(0, n_rows - 390)
            lf.layout.xaxis.range = [df.TIME_EST.iloc[start],
                                     df.TIME_EST.iloc[-1]]

# ────────────────────────────────────────────────────────────────────────
async def run_ws():
    ws = WebSocketClient(api_key=API_KEY,
                         feed=Feed.RealTime, market=Market.Stocks)
    ws.subscribe(*[f"A.{s}"  for s in TICKERS],   # 1-sec
                 *[f"AM.{s}" for s in TICKERS])   # 1-min
    await ws.connect(ws_handle)


In [ ]:
await run_ws()

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:83: UserWarning:

Converting A to a CSC (compressed sparse column) matrix; may take a while.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:113: UserWarning:

Converting P to a CSC (compressed sparse column) matrix; may take a while.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/cvxpy/problems/problem.py:1539: UserWarning:

Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:83: UserWarning:

Converting A to a CSC (compressed sparse column) matrix; may take a while.

/Users/User1/projects/anaconda3/envs/polars_test_env/lib/python3.9/site-packages/scs/__init__.py:113: UserWarning:

Converting P to a CSC (compressed sparse column) matri

CancelledError: 

In [ ]:
await run_ws()